<a href="https://colab.research.google.com/github/ShreehariA/twitter-data-analysis/blob/main/Colab_notebook/twitter_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#installing the pysentimiento library
pip install pysentimiento

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=a8d1739db169a3c5e9a43956431af1b0ef7227e135609fe834916359e4ee0c5f
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji


In [2]:
#importing the required functions from the library
from pysentimiento import create_analyzer
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import expit
from google.colab import drive
import pandas as pd
from google.colab import auth
import gspread
from google.auth import default
import statistics
from gspread_dataframe import set_with_dataframe

In [5]:
#authenticating with google sheets where the data is stored
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
#converting the sheet to a dataframe
worksheet = gc.open('all_tweets').sheet1
rows = worksheet.get_all_values()
df=pd.DataFrame.from_records(rows[2:], columns=rows[1])
display(df)

,Date,Screen Name,Full Name,Tweet Text,Media,Location,Retweets,Favorites,App,Followers,Follows,Verfied,User Since,Location,Bio,
0,26-08-2022 10:04:00,@TASKSHELP,EduPacers.,Get 24/7 support in: #Online class #Maths #Acc...,,,0,0,Twitter for Android,545,1148,,12-01-2020,"New York, USA",Kindly dm or email #edupacers@gmail.com for he...,
1,26-08-2022 10:03:00,@Peertechz,Peertechz Publications,Journal of Civil Engineering and Environmental...,https://pbs.twimg.com/media/FbD4CO-VQAAFd1U.jpg,,0,0,Twitter Web App,306,1898,,05-01-2014,Hyderabad,Peertechz Publications Pvt. Ltd. is an interna...,Request for research help
2,26-08-2022 04:30:00,@TheCrimeReport,The Crime Report,"""... existing #research into how victimized #c...",https://pbs.twimg.com/media/FbCrysKXEAAr4vn.jpg,,0,0,Buffer,16760,1615,,07-06-2009,United States,⚖️ The nation’s most comprehensive criminal ju...,
3,26-08-2022 10:02:00,@ReddySalina,Salina Reddy,#Swedish #political #parties have candidates #...,https://pbs.twimg.com/media/FbC5fRzWAAA4NjS.jpg,,0,0,Twitter Web App,499,1022,,11-04-2021,Earth,#Democracy is the face of a #layman Democracy ...,
4,26-08-2022 10:01:00,@MarketechAPAC,MARKETECH APAC,The research by @OracleHosp and @skift notes t...,,,0,0,Twitter Web App,143,28,,07-11-2020,Philippines,MARKETECH APAC is a marketing-centric publicat...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2954,24-08-2022 14:39:00,@corpus_news,CORPUS,#testbed #ai #graphcore #scientific #research ...,,,0,0,CORPUS_NEWS,738,420,,5/24/2022,"North Carolina, USA","With CORPUS, follow the latest in Artificial I...",
2955,24-08-2022 14:38:00,@HardmanandCo,Hardman & Co,$RECI #RealEstateCreditInvestments In this int...,,,0,2,Twitter Web App,20975,18334,,01-09-2012,London,For 25 years we have produced fundamental inve...,
2956,24-08-2022 14:38:00,@ReputableW,Reputable writers,Get legit and quality work always √ Pay #essay...,,,0,0,TweetDeck,875,1419,,2/27/2019,"Ohio,USA.Watsapp+1(929)3776634",We assist students write perfect assignments a...,
2957,24-08-2022 14:38:00,@ReputableW,Reputable writers,"HMU For quality, with in your budget and time ...",,,0,0,TweetDeck,875,1419,,2/27/2019,"Ohio,USA.Watsapp+1(929)3776634",We assist students write perfect assignments a...,


In [3]:
#initializing the sentiment analysis model
sentiment_analyzer = create_analyzer(task="sentiment", lang="en")
#initializing the topic analysis model
MODEL=f"cardiffnlp/tweet-topic-21-multi"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
class_mapping = model.config.id2label

config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [6]:
sentiment_list=[]
sentiment_scores=[]
topic_list=[]
for i in range(len(df)):
  #sentiment analysis
  sent_analyzer=sentiment_analyzer.predict(df['Tweet Text'][i])
  sent,prob="",0
  for key,value in sent_analyzer.probas.items():
    if value>0.2:
      sent+=","+key
      prob+=value
  sentiment_list.append(sent[1:])
  sentiment_scores.append(prob)

  #topic analysis
  tokens = tokenizer(df['Tweet Text'][i],return_tensors='pt')
  output = model(**tokens)
  scores = output[0][0].detach().numpy()
  scores = expit(scores)
  predictions = (scores >= 0.5) * 1
  topic=""
  for i in range(len(predictions)):
    if predictions[i]:
      topic+=","+class_mapping[i]
  topic_list.append(topic[1:])

#getting the prob score
print(statistics.mean(sentiment_scores))
df['sentiment']=sentiment_list
df['prob_score']=sentiment_scores
df['topic']=topic_list

0.9453422437803131


In [7]:
display(df)

,Date,Screen Name,Full Name,Tweet Text,Media,Location,Retweets,Favorites,App,Followers,Follows,Verfied,User Since,Location,Bio,,sentiment,prob_score,topic
0,26-08-2022 10:04:00,@TASKSHELP,EduPacers.,Get 24/7 support in: #Online class #Maths #Acc...,,,0,0,Twitter for Android,545,1148,,12-01-2020,"New York, USA",Kindly dm or email #edupacers@gmail.com for he...,,"NEU,POS",0.997320,"learning_&_educational,youth_&_student_life"
1,26-08-2022 10:03:00,@Peertechz,Peertechz Publications,Journal of Civil Engineering and Environmental...,https://pbs.twimg.com/media/FbD4CO-VQAAFd1U.jpg,,0,0,Twitter Web App,306,1898,,05-01-2014,Hyderabad,Peertechz Publications Pvt. Ltd. is an interna...,Request for research help,NEU,0.976771,"learning_&_educational,news_&_social_concern,s..."
2,26-08-2022 04:30:00,@TheCrimeReport,The Crime Report,"""... existing #research into how victimized #c...",https://pbs.twimg.com/media/FbCrysKXEAAr4vn.jpg,,0,0,Buffer,16760,1615,,07-06-2009,United States,⚖️ The nation’s most comprehensive criminal ju...,,NEG,0.808465,news_&_social_concern
3,26-08-2022 10:02:00,@ReddySalina,Salina Reddy,#Swedish #political #parties have candidates #...,https://pbs.twimg.com/media/FbC5fRzWAAA4NjS.jpg,,0,0,Twitter Web App,499,1022,,11-04-2021,Earth,#Democracy is the face of a #layman Democracy ...,,"NEG,NEU",0.994499,news_&_social_concern
4,26-08-2022 10:01:00,@MarketechAPAC,MARKETECH APAC,The research by @OracleHosp and @skift notes t...,,,0,0,Twitter Web App,143,28,,07-11-2020,Philippines,MARKETECH APAC is a marketing-centric publicat...,,"NEU,POS",0.995693,"business_&_entrepreneurs,news_&_social_concern..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2954,24-08-2022 14:39:00,@corpus_news,CORPUS,#testbed #ai #graphcore #scientific #research ...,,,0,0,CORPUS_NEWS,738,420,,5/24/2022,"North Carolina, USA","With CORPUS, follow the latest in Artificial I...",,NEU,0.961153,science_&_technology
2955,24-08-2022 14:38:00,@HardmanandCo,Hardman & Co,$RECI #RealEstateCreditInvestments In this int...,,,0,2,Twitter Web App,20975,18334,,01-09-2012,London,For 25 years we have produced fundamental inve...,,NEU,0.960611,"business_&_entrepreneurs,news_&_social_concern"
2956,24-08-2022 14:38:00,@ReputableW,Reputable writers,Get legit and quality work always √ Pay #essay...,,,0,0,TweetDeck,875,1419,,2/27/2019,"Ohio,USA.Watsapp+1(929)3776634",We assist students write perfect assignments a...,,"NEU,POS",0.997385,learning_&_educational
2957,24-08-2022 14:38:00,@ReputableW,Reputable writers,"HMU For quality, with in your budget and time ...",,,0,0,TweetDeck,875,1419,,2/27/2019,"Ohio,USA.Watsapp+1(929)3776634",We assist students write perfect assignments a...,,NEU,0.966293,"learning_&_educational,youth_&_student_life"


In [14]:
#initializing the sentiment analysis as new dict
dict_sent_Retweets={'NEG':0,'POS':0,'NEU':0}
dict_sent_Favorites={'NEG':0,'POS':0,'NEU':0}
dict_sent_Followers={'NEG':0,'POS':0,'NEU':0}
#initializing the topic analysis as new dict
dict_topic_Retweets={
    'arts_&_culture': 0,
    'business_&_entrepreneurs': 0,
    'celebrity_&_pop_culture': 0,
    'diaries_&_daily_life': 0,
    'family': 0,
    'fashion_&_style': 0,
    'film_tv_&_video': 0,
    'fitness_&_health': 0,
    'food_&_dining': 0,
    'gaming': 0,
    'learning_&_educational': 0,
    'music': 0,
    'news_&_social_concern': 0,
    'other_hobbies': 0,
    'relationships': 0,
    'science_&_technology': 0,
    'sports': 0,
    'travel_&_adventure': 0,
    'youth_&_student_life': 0,
    'unknown':0
}
dict_topic_Favorites={
    'arts_&_culture': 0,
    'business_&_entrepreneurs': 0,
    'celebrity_&_pop_culture': 0,
    'diaries_&_daily_life': 0,
    'family': 0,
    'fashion_&_style': 0,
    'film_tv_&_video': 0,
    'fitness_&_health': 0,
    'food_&_dining': 0,
    'gaming': 0,
    'learning_&_educational': 0,
    'music': 0,
    'news_&_social_concern': 0,
    'other_hobbies': 0,
    'relationships': 0,
    'science_&_technology': 0,
    'sports': 0,
    'travel_&_adventure': 0,
    'youth_&_student_life': 0,
    'unknown':0
}
dict_topic_Followers={
    'arts_&_culture': 0,
    'business_&_entrepreneurs': 0,
    'celebrity_&_pop_culture': 0,
    'diaries_&_daily_life': 0,
    'family': 0,
    'fashion_&_style': 0,
    'film_tv_&_video': 0,
    'fitness_&_health': 0,
    'food_&_dining': 0,
    'gaming': 0,
    'learning_&_educational': 0,
    'music': 0,
    'news_&_social_concern': 0,
    'other_hobbies': 0,
    'relationships': 0,
    'science_&_technology': 0,
    'sports': 0,
    'travel_&_adventure': 0,
    'youth_&_student_life': 0,
    'unknown':0
}

In [15]:
#getting the necessary data in the dict
for i in range(len(df)):
  sent_list=df['sentiment'][i].split(',')
  topic_list=df['topic'][i].split(',')
  for s in sent_list:
    dict_sent_Retweets[s]+=int(df['Retweets'][i])
    dict_sent_Favorites[s]+=int(df['Favorites'][i])
    dict_sent_Followers[s]+=int(df['Followers'][i])
  for t in topic_list:
    if len(t)>1:
      dict_topic_Retweets[t]+=int(df['Retweets'][i])
      dict_topic_Favorites[t]+=int(df['Favorites'][i])
      dict_topic_Followers[t]+=int(df['Followers'][i])
    else:
      #creating a unknown topic which cannot be identified by the topic analysis
      dict_topic_Retweets['unknown']+=int(df['Retweets'][i])
      dict_topic_Favorites['unknown']+=int(df['Favorites'][i])
      dict_topic_Followers['unknown']+=int(df['Followers'][i])

print(dict_sent_Retweets)
print(dict_topic_Retweets)

{'NEG': 55, 'POS': 903, 'NEU': 1100}
{'arts_&_culture': 9, 'business_&_entrepreneurs': 179, 'celebrity_&_pop_culture': 0, 'diaries_&_daily_life': 14, 'family': 0, 'fashion_&_style': 2, 'film_tv_&_video': 0, 'fitness_&_health': 185, 'food_&_dining': 6, 'gaming': 1, 'learning_&_educational': 745, 'music': 2, 'news_&_social_concern': 356, 'other_hobbies': 0, 'relationships': 0, 'science_&_technology': 467, 'sports': 5, 'travel_&_adventure': 6, 'youth_&_student_life': 524, 'unknown': 14}


In [34]:
#converting the sentement analysis dict to df and joining them to give a bigger picture
analysis_sent_df=pd.DataFrame(columns=['sentement','Retweets','Favorites','Followers'])
analysis_sent_df['sentement']=list(dict_sent_Retweets.keys())
analysis_sent_df['Retweets']=list(dict_sent_Retweets.values())
analysis_sent_df['Favorites']=list(dict_sent_Favorites.values())
analysis_sent_df['Followers']=list(dict_sent_Followers.values())
display(analysis_sent_df)

#converting the topic analysis dict to df and joining them to give a bigger picture
analysis_topic_df=pd.DataFrame(columns=['topic','Retweets','Favorites','Followers'])
analysis_topic_df['topic']=list(dict_topic_Retweets.keys())
analysis_topic_df['Retweets']=list(dict_topic_Retweets.values())
analysis_topic_df['Favorites']=list(dict_topic_Favorites.values())
analysis_topic_df['Followers']=list(dict_topic_Followers.values())
display(analysis_topic_df)

,sentement,Retweets,Favorites,Followers
0,NEG,55,119,7111526
1,POS,903,1353,6786384
2,NEU,1100,1425,18782561


,topic,Retweets,Favorites,Followers
0,arts_&_culture,9,45,94078
1,business_&_entrepreneurs,179,353,2257704
2,celebrity_&_pop_culture,0,1,699
3,diaries_&_daily_life,14,46,6637025
4,family,0,0,917
5,fashion_&_style,2,1,5544
6,film_tv_&_video,0,3,46996
7,fitness_&_health,185,361,10272989
8,food_&_dining,6,9,249494
9,gaming,1,2,2372


from matplotlib import pyplot as plt
analysis_sent_df['Retweets'].plot(kind='hist', bins=20, title='Retweets')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
analysis_sent_df['Favorites'].plot(kind='hist', bins=20, title='Favorites')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
analysis_sent_df['Followers'].plot(kind='hist', bins=20, title='Followers')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
analysis_sent_df['Favorites'].plot(kind='line', figsize=(8, 4), title='Favorites')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
analysis_sent_df['Followers'].plot(kind='line', figsize=(8, 4), title='Followers')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(analysis_sent_df['sentement'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(analysis_sent_df, x='Retweets', y='sentement', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(analysis_sent_df['sentement'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(analysis_sent_df, x='Favorites', y='sentement', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(analysis_sent_df['sentement'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(analysis_sent_df, x='Followers', y='sentement', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [39]:
#creating a new sheet to store the results
sh = gc.create('sentiment_results')
spreadsheet = gc.open('sentiment_results')
#storing the results in individual sheets
analysis_worksheet = spreadsheet.add_worksheet(title='analysis_added', rows='100', cols='100')
set_with_dataframe(analysis_worksheet, df)
sent_worksheet = spreadsheet.add_worksheet(title='sentiment_analysis', rows='100', cols='100')
set_with_dataframe(sent_worksheet, analysis_sent_df)
topic_worksheet = spreadsheet.add_worksheet(title='topic_analysis', rows='100', cols='100')
set_with_dataframe(topic_worksheet, analysis_topic_df)